<b>RNN</b>

- Very powerful 
- Requires Data
- Tricky to train (somewhat unstable)

- Are Neural networks that contains a recurrent layer
- can sequencially process a sequence of input
- Three dimensional input
![Alt text](explanation_images/rnns.png)

- Single memory cell(Small NN) used repeatdly to compute the outputs.
- Single memory cell could be:
    - Simple Dense Layer
    - Complex: LSTM cell, GRU cell
 
Only single cell, in the diagram it's the same cell

- X0: input
- Y0 hat: current output
- H0: state or context factor, it's fed to the memory cell 

![Alt text](explanation_images/singlecCell.png)

![Alt text](image.png)

![Alt text](image-1.png)